# Data Processing

This files contains the code to process the streamingqa dataset

In [1]:
ls /home/saisur/streaming_qa

filtered/              streaminqa_train.jsonl  wmt_sorting_key_ids.txt
streaminqa_eval.jsonl  streaminqa_valid.jsonl


In [1]:
import glob
files_qa = glob.glob("/home/saisur/streaming_qa/*.jsonl")
files_qa


['/home/saisur/streaming_qa/streaminqa_train.jsonl',
 '/home/saisur/streaming_qa/streaminqa_eval.jsonl',
 '/home/saisur/streaming_qa/streaminqa_valid.jsonl']

In [2]:
from datasets import load_dataset
dataset = load_dataset('json', data_files=files_qa[0])
valid_dataset = load_dataset('json', data_files=files_qa[1])
test_dataset = load_dataset('json', data_files=files_qa[2])
dataset["validation"] = valid_dataset["train"]
dataset["test"] = test_dataset["train"]
del valid_dataset, test_dataset
dataset

Using custom data configuration default-9abcb2bb7b1156aa
Found cached dataset json (/home/saisur/.cache/huggingface/datasets/json/default-9abcb2bb7b1156aa/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-4d5fe5e1c684f3c2
Found cached dataset json (/home/saisur/.cache/huggingface/datasets/json/default-4d5fe5e1c684f3c2/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-3defb22cdfd0db83
Found cached dataset json (/home/saisur/.cache/huggingface/datasets/json/default-3defb22cdfd0db83/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['qa_id', 'question', 'answers', 'answers_additional', 'question_ts', 'evidence_ts', 'evidence_id', 'recent_or_past', 'written_or_generated', 'toxicity_identity_attack', 'toxicity_insult', 'toxicity_profanity', 'toxicity_severe_toxicity', 'toxicity_sexually_explicit', 'toxicity_threat'],
        num_rows: 99402
    })
    validation: Dataset({
        features: ['qa_id', 'question', 'answers', 'answers_additional', 'question_ts', 'evidence_ts', 'evidence_id', 'recent_or_past', 'written_or_generated', 'toxicity_identity_attack', 'toxicity_insult', 'toxicity_profanity', 'toxicity_severe_toxicity', 'toxicity_sexually_explicit', 'toxicity_threat'],
        num_rows: 36378
    })
    test: Dataset({
        features: ['qa_id', 'question', 'answers', 'answers_additional', 'question_ts', 'evidence_ts', 'evidence_id', 'recent_or_past', 'written_or_generated', 'toxicity_identity_attack', 'toxicity_insult', 'toxicity_profanity', 'toxicity_seve

In [3]:
train_evidemceid = dataset["train"]["evidence_id"]
test_evidemceid = dataset["test"]["evidence_id"]
valid_evidemceid = dataset["validation"]["evidence_id"]
all_evidence = train_evidemceid + test_evidemceid + valid_evidemceid
all_evidence = set([x.split("\x00\x01")[1] for x in all_evidence])
all_evidence = set(all_evidence)

In [4]:
for i in all_evidence:
    print(i)
    break

b6e394c8ace3b529e2c65d87ad952729d97af4335e83482a646eedec256ef97c


In [5]:
len(all_evidence[0])

TypeError: 'set' object is not subscriptable

In [6]:
location_hashes = "/home/saisur/streaming_qa/wmt_sorting_key_ids.txt"
with open(location_hashes, "r") as f:
    hashes = f.readlines()
hashes = set([x.strip().split("\x00\x01")[1] for x in hashes])

In [7]:
hashes

{'...',
 ...}

In [8]:
len(hashes.intersection(set(all_evidence)))

138502

In [9]:
all_raw = glob.glob("/home/saisur/data/*.filtered")

In [10]:
raw_datasets = load_dataset('csv', data_files=all_raw,  column_names=["date", "sentence_split", "non_split"], delimiter="\t")

Using custom data configuration default-58cde7c1770b75d0
Found cached dataset csv (/home/saisur/.cache/huggingface/datasets/csv/default-58cde7c1770b75d0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['date', 'sentence_split', 'non_split'],
        num_rows: 19863186
    })
})

In [ ]:
raw_datasets_non_split = raw_datasets["train"]["non_split"]

In [12]:
# Write a function which takes sting and hashes it with sha256
import hashlib
def hash_string(string):
    return hashlib.sha256(string.encode()).hexdigest()

def hash_list(example):
    list_of_strings = example["non_split"]
    example["hash"] = [hash_string(x) for x in list_of_strings]    
    return example

raw_datasets = raw_datasets.map(hash_list, batched=True, num_proc=46)

Loading cached processed dataset at /home/saisur/.cache/huggingface/datasets/csv/default-58cde7c1770b75d0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-fad28191870ac572.arrow


Loading cached processed dataset at /home/saisur/.cache/huggingface/datasets/csv/default-58cde7c1770b75d0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-285e6c0313dc4c5a.arrow


Loading cached processed dataset at /home/saisur/.cache/huggingface/datasets/csv/default-58cde7c1770b75d0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-1ee91e18c1638ee8.arrow


Loading cached processed dataset at /home/saisur/.cache/huggingface/datasets/csv/default-58cde7c1770b75d0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-904e61a5cc6d4afc.arrow


Loading cached processed dataset at /home/saisur/.cache/huggingface/datasets/csv/default-58cde7c1770b75d0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-bb966843fbe7df24.arrow


...

Loading cached processed dataset at /home/saisur/.cache/huggingface/datasets/csv/default-58cde7c1770b75d0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-428024403764fac6.arrow


Loading cached processed dataset at /home/saisur/.cache/huggingface/datasets/csv/default-58cde7c1770b75d0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-8c4707a226c3d70e.arrow


Loading cached processed dataset at /home/saisur/.cache/huggingface/datasets/csv/default-58cde7c1770b75d0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-f82ad15a38471b45.arrow


Loading cached processed dataset at /home/saisur/.cache/huggingface/datasets/csv/default-58cde7c1770b75d0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-b5608b045b9f851e.arrow


Loading cached processed dataset at /home/saisur/.cache/huggingface/datasets/csv/default-58cde7c1770b75d0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-3ee7be9b955cd22a.arrow
Loading cached processed dataset at /home/saisur/.cache/huggingface/datasets/csv/default-58cde7c1770b75d0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-7251293f14d0d6fc.arrow


In [13]:
raw_datasets["train"][90]

{'date': 20080101,
 'sentence_split': 'VGVib3cgRmFjZXMgVG91Z2ggRGVmZW5zZSwgYW5kIERvdWJ0ZXJzCk9STEFORE8sIEZsYS4gLSBRdWFydGVyYmFjayBUaW0gVGVib3cgb2YgRmxvcmlkYSB3b24gdGhlIEhlaXNtYW4gVHJvcGh5IHRoaXMgc2Vhc29uLCB0aGUgb25seSBzb3Bob21vcmUgZXZlciB0byBkbyBzby4KU2hhd24gQ3JhYmxlLCBhIHNlbmlvciBNaWNoaWdhbiBsaW5lYmFja2VyLCBkb2VzIG5vdCBzZWVtIHRlcnJpYmx5IGltcHJlc3NlZC4KIllvdSBhbGwgYXJlIHRvbyBjb25jZXJuZWQgd2l0aCB0aGlzIHdob2xlIFRlYm93IGh5cGUsIiBDcmFibGUgdG9sZCByZXBvcnRlcnMgYXMgaGlzIFdvbHZlcmluZXMgcHJlcGFyZWQgdG8gcGxheSBUZWJvdydzIEdhdG9ycyBvbiBUdWVzZGF5IGluIHRoZSBDYXBpdGFsIE9uZSBCb3dsLgpJIG1lYW4sIGhlJ3MganVzdCBhIHF1YXJ0ZXJiYWNrLgpIZSdzIG5vdGhpbmcgbW9yZSBleGNlcHRpb25hbCB0aGFuIHdlJ3ZlIGV2ZXIgcGxheWVkIGFnYWluc3QuCkFuZCB3aGF0IGFib3V0IFRlYm93J3MgMjkgcGFzc2luZyB0b3VjaGRvd25zIGFuZCAyMiBydXNoaW5nIHRvdWNoZG93bnMgYXMgaGUgbGVkIGhpcyB0ZWFtIHRvIGEgOS0zIHJlY29yZD8KVGhleSBhcmUganVzdCAiZnJlYWtpc2ggc3RhdGlzdGljcywiIENyYWJsZSBzYWlkLgoiVGhleSB1c2UgaGltIGRpZmZlcmVudGx5LCB0aGV5IHVzZSBoaW0gYSBsb3QgdG8gcnVuIHRoZSBiYWxsIGluLCIgQ

In [14]:
hashes

{'...',
 ...}

In [16]:
# hashes_filtered = raw_datasets["train"]["hash"]

In [17]:
# hashes_filtered = set(hashes_filtered)

In [39]:
len(hashes_filtered)

13586630

In [50]:
len(hashes_filtered.intersection(hashes))

11393471

In [51]:
len(hashes_filtered.intersection(all_evidence))

138502

In [18]:
filtered_ds_large = raw_datasets.filter(lambda x: x["hash"] in hashes, num_proc=46)

#0:   0%|          | 0/432 [00:00<?, ?ba/s]

#2:   0%|          | 0/432 [00:00<?, ?ba/s]

#3:   0%|          | 0/432 [00:00<?, ?ba/s]

#1:   0%|          | 0/432 [00:00<?, ?ba/s]

#4:   0%|          | 0/432 [00:00<?, ?ba/s]

...

#41:   0%|          | 0/432 [00:00<?, ?ba/s]

#42:   0%|          | 0/432 [00:00<?, ?ba/s]

#43:   0%|          | 0/432 [00:00<?, ?ba/s]

#45:   0%|          | 0/432 [00:00<?, ?ba/s]

#44:   0%|          | 0/432 [00:00<?, ?ba/s]

In [19]:
filtered_ds_large

DatasetDict({
    train: Dataset({
        features: ['date', 'sentence_split', 'non_split', 'hash'],
        num_rows: 15941008
    })
})

In [56]:
filtered_ds.save_to_disk("/home/saisur/data/filtered_small")

Flattening the indices:   0%|          | 0/178 [00:00<?, ?ba/s]

In [54]:
all_evidence

{'...',
 ...}

In [7]:
import hashlib
#encode the non_split as sha256
raw_datasets["train"] = raw_datasets.map(lambda x: {"hash": hashlib.sha256(x["non_split"].encode()).hexdigest()}, num_proc=46)

#0:   0%|          | 0/431809 [00:00<?, ?ex/s]

#1:   0%|          | 0/431809 [00:00<?, ?ex/s]

#2:   0%|          | 0/431809 [00:00<?, ?ex/s]

#3:   0%|          | 0/431809 [00:00<?, ?ex/s]

#4:   0%|          | 0/431809 [00:00<?, ?ex/s]

...

#41:   0%|          | 0/431808 [00:00<?, ?ex/s]

#42:   0%|          | 0/431808 [00:00<?, ?ex/s]

#43:   0%|          | 0/431808 [00:00<?, ?ex/s]

#44:   0%|          | 0/431808 [00:00<?, ?ex/s]

#45:   0%|          | 0/431808 [00:00<?, ?ex/s]

In [ ]:
from tqdm import tqdm 
for i in tqdm(range(100)):
    continue

100%|██████████| 100/100 [00:00<00:00, 1402777.26it/s]
